&emsp;&emsp;易知,跳字模型条件概率对应的对数损失为:

$$ -\log P\left(w_{o} \mid w_{c}\right)=-\mathbf{u}_{o}^{\top} \boldsymbol{v}_{c}+\log \left(\sum_{i \in \mathcal{V}} \exp \left(\mathbf{u}_{i}^{\top} \mathbf{v}_{c}\right)\right) $$

其中$ \mathbf{u}_o$为背景词向量$w_o$的词向量,$\mathbf{v}_c$为中心词$w_c$的词向量,$\mathbf{u}_i$为任意某个词向量.

&emsp;&emsp;由于softmax运算考虑了背景词可能是词典$V$中的任一词,以上损失包含了词典大小数目的项的累加.不论是跳字模型还是连续词袋模型,由于条件概率使用了softmax运算,
每一步的梯度计算都包含词典大小数目的项的累加.对于含几十万或上百万词的较大词典,每次的梯度计算开销可能过大.为了降低该计算复杂度,下面介绍一种近似训练方法,即负采样(negative sampling).

&emsp;&emsp;负采样修改了原来的目标函数,给定中心词的一个背景窗口,将背景词$w_o$出现在该背景窗口看作一个事件,并将事件的概率计算为

$$ P\left(D=1 \mid w_{c}, w_{o}\right)=\sigma\left(\mathbf{u}_{o}^{\top} \mathbf{v}_{c}\right) $$

其中,$ \sigma $与sigmoid激活函数定义相同:

$$ \sigma(x) = \frac{1}{1+\exp(-x)} $$

若只考虑最大化文本序列中所有该事件的联合概率来训练词向量.也就是,给定一个长度为$T$,设时间步$t$的词为$w_t$且背景窗口为$m$,考虑最大化联合概率

$$ \prod_{t=1}^{T} \prod_{-m \leq j \leq m, j \neq 0} P\left(D=1 \mid w_{t}, w_{t+j}\right) $$

然而,以上模型中包含的事件仅考虑了正类样本.这导致当所有词向量相等且值为无穷大时,以上的联合概率才被最大化为1.
很明显,这样的词向量毫无意义.负采样通过采样并添加负类样本使目标函数更有意义.设背景词$w_o$出现在中心词$w_{c}$的一个背景窗口为事件$P$,
我们根据分布$P(w)$采样$K$个未出现在该背景窗口中的词,即噪声词.设噪声词$w_{k}(k=1, \ldots, K)$不出现在中心词$w_{c}$的该背景窗口为事件$N_{k}$.
假设同时含有正类样本和负类样本的事件$P, N_{1}, \ldots, N_{K}$相互独立,负采样将以上需要最大化的仅考虑正类样本的联合概率改写为

$$ \prod_{t=1}^{T} \prod_{-m \leq j \leq m, j \neq 0} P\left(w_{t+j} \mid w_{t}\right)  $$

其中条件概率被<font color='red'>近似</font>表示为:

$$ P\left(w_{t+j} \mid w_{t}\right)=P\left(D=1 \mid w_{t}, w_{t+j}\right) \prod_{k=1, w_{k} \sim P(w)}^{K} P\left(D=0 \mid w_{t}, w_{k}\right) $$

设文本序列中时间步$t$的词$w_t$在词典的索引为$ i_t$,噪音词$w_k$在词典中的索引为$h_k$,有关以上条件概率的对数损失为

$$
\begin{aligned}
-\log P\left(w_{t+j} \mid w_{t}\right) &=-\log P\left(D=1 \mid w_{t}, w_{t+j}\right)-\sum_{k=1, w_{k} \sim P(w)}^{K} \log P\left(D=0 \mid w_{t}, w_{k}\right) \\
&=-\log \sigma\left(\mathbf{u}_{i_{t+j}}^{\top} \mathbf{v}_{i_{t}}\right)-\sum_{k=1, w_{k} \sim P(w)}^{K} \log \left(1-\sigma\left(\mathbf{u}_{h_{k}}^{\top} \mathbf{v}_{i_{t}}\right)\right) \\
&=-\log \sigma\left(\mathbf{u}_{i_{t+j}}^{\top} \mathbf{v}_{i_{t}}\right)-\sum_{k=1}^{K} \log \sigma\left(-\mathbf{u}_{h_{k}}^{\top} \mathbf{v}_{i_{t}}\right) \qquad \text{注:利用了等式$1 - \sigma{(x)} = \sigma{(-x)}$}
\end{aligned}
$$

其中$ \mathbf{v}_{i_{t}} $为中心词词向量,$ \mathbf{u}_{i_{t+j}} $为背景词词向量,$\mathbf{u}_{h_{k}}$为噪音词词向量.

可以看出,最大化$\log P(w_{t+j})$相当于最大化$ \sigma\left(\mathbf{u}_{i_{t+j}}^{\top} \mathbf{v}_{i_{t}}\right) $,同时最小化$ \sigma\left(\mathbf{u}_{h_{k}}^{\top} \mathbf{v}_{i_{t}}\right) $,
这恰好是我们所希望的,即增大正样本概率的同时降低负样本的概率.现在,训练中每一步的梯度计算开销不再与词典大小相关,而与K线性相关.
当K取较小的常数时,负采样在每一步的梯度计算开销较小.CBOW的负采样方法与之基本类似.